In [5]:
import tensorflow as tf
#from tensorflow.examples.tutorials.mnist import input_data
#import tensorflow.compat.v1 as tf_v1
#tf_v1.disable_eager_execution()
#x = tf_v1.placeholder(tf.float32, shape=(None, 3), name='x')
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import numpy as np

#mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
#x_image = np.concatenate((x_train,x_test),axis=0)
#xs = tf.reshape(x_image,[1,28*28])
ys = np.concatenate((y_train,y_test),axis=0)

learning_rate = 1e-4
keep_prob = 0.7 # 
max_epoch = 2000
batch_size = 32
num_batches =  len(x_train) // batch_size

#def compute_accuracy(v_xs, v_ys):
#    #global prediction
#    y_pre = sess.run(prediction, feed_dict={xs: v_xs, keep_prob: 1})
#    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
#    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys, keep_prob: 1})
#    return result

def compute_accuracy(v_xs, v_ys):
    y_pre = model(v_xs)
    correct_prediction = tf.equal(tf.argmax(y_pre, 1), tf.argmax(v_ys, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy.numpy()

def weight_variable(shape):
    initial = tf.random.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    # 每一维度  滑动步长全部是 1， padding 方式 选择 same
    # 提示 使用函数  tf.nn.conv2d
    
    return tf.nn.conv2d(input=x,filters=W,strides=[1,1,1,1],padding='SAME')

def max_pool_2x2(x):
    # 滑动步长 是 2步; 池化窗口的尺度 高和宽度都是2; padding 方式 请选择 same
    # 提示 使用函数  tf.nn.max_pool
    
    return tf.nn.max_pool(input=x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')


#xs = tf_v1.placeholder(tf.float32, [None, 784])/255.
#ys = tf_v1.placeholder(tf.float32, [None, 10])
#keep_prob = tf_v1.placeholder(tf.float32)
#x_image = tf.reshape(xs, [-1, 28, 28, 1])

#  卷积层 1
## conv1 layer ##
W_conv1 = weight_variable([7,7,1,32]) # patch 7x7, in size 1, out size 32
b_conv1 = weight_variable([32])                     
# 卷积层 2
W_conv2 = weight_variable([5,5,32,64])                       # patch 5x5, in size 32, out size 64
b_conv2 = weight_variable([64])
#  全连接层 1
## fc1 layer ##
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])
# 全连接层 2
## fc2 layer ##
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

def model(x):
    h_conv1 = tf.nn.relu(conv2d(x,W_conv1)+b_conv1) # 卷积  自己选择 选择激活函数
    h_pool1 = max_pool_2x2(h_conv1)                      # 池化               
    h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2)                       # 卷积  自己选择 选择激活函数
    h_pool2 = max_pool_2x2(h_conv2)                       # 池化
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
    return tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
    

# 交叉熵函数
#cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.math.log(prediction),axis=[1]))
# 需要更新的参数
#var_list = [W_conv1,b_conv1,W_conv2,b_conv2,W_fc1,b_fc1,W_fc2,b_fc2]
#train_step = tf.keras.optimizers.Adam(learning_rate).minimize(cross_entropy,var_list)

#with tf.Session() as sess:
#    init = tf.global_variables_initializer()
#    sess.run(init)
    
#    for i in range(max_epoch):
#        batch_xs, batch_ys = mnist.train.next_batch(100)
#        sess.run(train_step, feed_dict={xs: batch_xs, ys: batch_ys, keep_prob:keep_prob_rate})
#        if i % 100 == 0:
#            print(compute_accuracy(
#                mnist.test.images[:1000], mnist.test.labels[:1000]))

optimizer = tf.keras.optimizers.Adam(learning_rate)


# 训练模型
#for i in range(max_epoch):
#    batch_xs, batch_ys = mnist.train.next_batch(100)
#    batch_ys = tf.cast(batch_ys, dtype=tf.float32)
#    loss = train_step(batch_xs, batch_ys)
#    if i % 100 == 0:
#        print(compute_accuracy(mnist.test.images[:1000], mnist.test.labels[:1000]))



In [6]:
@tf.function
def train_step(xs, ys):
    with tf.GradientTape() as tape:        
        #prediction = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
        prediction = model(xs,)
        cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.math.log(prediction), axis=[1]))
    var_list = [W_conv1,b_conv1,W_conv2,b_conv2,W_fc1,b_fc1,W_fc2,b_fc2]
    gradients = tape.gradient(cross_entropy, var_list)
    optimizer.apply_gradients(zip(gradients, var_list))
    return cross_entropy


In [ ]:
for i in range(max_epoch):
    for batch in range(num_batches):
        start_idx = batch * batch_size
        end_idx = start_idx + batch_size
        batch_xs = x_train[start_idx:end_idx]
        batch_ys = y_train[start_idx:end_idx]
        loss = train_step(batch_xs, batch_ys)
        #print(compute_accuracy(x_test[:1000], y_test[:1000]))

In [4]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
